In [1]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
from prod_details import product_demand_per_dc, handling_out_costs_per_dc, total_operational_costs

# from prod_details import product_demand_per_dc, handling_out_costs_per_dc, shipping_costs_per_dc

In [5]:
# Load the data
outbound_data = pd.read_excel("Outbound.xlsx")
demand_data = pd.read_excel('Demand Forecast.xlsx')
unit_data = pd.read_excel('Product Data per State.xlsx')
oc_data = pd.read_excel('Opening-Closing Costs.xlsx')

warehousing_handlingOut_data = pd.read_excel('Warehousing.xlsx', sheet_name='Handling Out')
warehousing_handlingOut_data.index = warehousing_handlingOut_data.DC
outbound_data = outbound_data.drop(['State','Small shipment'], axis=1)

# Define the index of some data to simplify future data indexing.
outbound_data.index = outbound_data.State
demand_data.index = demand_data.state
unit_data.index = unit_data.state
oc_data.index = oc_data.State

as_is_dc = {
    "NY": ['CT', 'DC', 'DE', 'MA', 'MD', 'ME', 'NH', 'NJ', 'NY', 'PA', 'RI', 'VT'],
    "ND": ['MN', 'MT', 'ND', 'SD', 'WY'],
    "IL": ['HI', 'IA', 'IL', 'IN', 'KS', 'KY', 'MI', 'MO', 'NE', 'OH', 'VA', 'WI', 'WV'],
    "TN": ['AL', 'FL', 'GA', 'NC', 'SC', 'TN'],
    "WA": ['AK', 'ID', 'OR', 'WA'],
    "TX": ['AR', 'AZ', 'LA', 'MS', 'NM', 'OK', 'TX'],
    "UT": ['CA', 'CO', 'NV', 'UT']}

AttributeError: 'DataFrame' object has no attribute 'State'

In [3]:
# As-Is situation

def outbound_costs(dc_allocation):

    dc_handling_out_costs = sum(handling_out_costs_per_dc(dc_allocation, unit_data, warehousing_handlingOut_data))

    shipping_costs = []
    for state in demand_data.index:
        for dc, states in dc_allocation.items():
            if state in states:
                total_prod_weight_per_state = demand_data['total_weight'][state]
                tariff_from_dc_to_state = outbound_data.loc[state, dc]
                shipping_costs.append(total_prod_weight_per_state * tariff_from_dc_to_state)

    dc_shipping_costs = sum(shipping_costs)
    
    total_outbound_costs = dc_handling_out_costs + dc_shipping_costs
    print('Total Shipping Costs (from DC to Customers):', dc_shipping_costs)
    print('Total Handling Out Costs:', dc_handling_out_costs)
    print('Total Outbound Costs:', total_outbound_costs)
    
    return total_outbound_costs

In [4]:
# As-Is Situation

# Define the as-is DC allocation. The keys are the DC, the values are the states where the corresponding DC's customers are.
as_is_dc = {
    "NY": ['CT', 'DC', 'DE', 'MA', 'MD', 'ME', 'NH', 'NJ', 'NY', 'PA', 'RI', 'VT'],
    "ND": ['MN', 'MT', 'ND', 'SD', 'WY'],
    "IL": ['HI', 'IA', 'IL', 'IN', 'KS', 'KY', 'MI', 'MO', 'NE', 'OH', 'VA', 'WI', 'WV'],
    "TN": ['AL', 'FL', 'GA', 'NC', 'SC', 'TN'],
    "WA": ['AK', 'ID', 'OR', 'WA'],
    "TX": ['AR', 'AZ', 'LA', 'MS', 'NM', 'OK', 'TX'],
    "UT": ['CA', 'CO', 'NV', 'UT']
}

# Calculate the total outbound_costs
outbound_costs(as_is_dc)
total_operational_costs(as_is_dc)

NameError: name 'unit_data' is not defined

In [47]:
# Algorithm 1: Greedy Constructive Search
# Allocate dc based on the distance/costs greedily 
greedy_dc_allocation = {}
for state in outbound_data.index:
    best_dc = (outbound_data.loc[state]).idxmin()
    
    if best_dc not in greedy_dc_allocation:
        greedy_dc_allocation[best_dc] = [state]
    elif isinstance(greedy_dc_allocation[best_dc], list):
        greedy_dc_allocation[best_dc].append(state)
    else:
        greedy_dc_allocation[best_dc] = [greedy_dc_allocation[best_dc], state]

# Calculate the outbound costs
outbound_costs(greedy_dc_allocation)

# Turns out, even though the shipping costs are lower,
# the total outbound costs are relatively higher since the operating and opening costs are skyrocketed.

Total Shipping Costs (from DC to Customers): 911216.61240381
Total Handling Out Costs: 1825572.3299999996
Total Outbound Costs: 2736788.9424038096


2736788.9424038096

In [60]:
# Algorithm 1b: Greedy Constructive Heuristics
# Now, knowing that opening and operating new DC can be costly, we apply the same algorithm without opening a new DC.

greedy_dc_allocation = {}
for state in outbound_data.index:
    current_dc = outbound_data[list(as_is_dc.keys())]
    best_dc = (current_dc.loc[state]).idxmin()
    if best_dc not in as_is_dc:
        print(best_dc)
    
    if best_dc not in greedy_dc_allocation:
        greedy_dc_allocation[best_dc] = [state]
    elif isinstance(greedy_dc_allocation[best_dc], list):
        greedy_dc_allocation[best_dc].append(state)
    else:
        greedy_dc_allocation[best_dc] = [greedy_dc_allocation[best_dc], state]

outbound_costs(greedy_dc_allocation)

# The shipping costs are lowered again, but the handling costs are increasing slightly.
# It resulted in a less significant decrease.

Total Shipping Costs (from DC to Customers): 935290.7478927901
Total Handling Out Costs: 1892668.1400000001
Total Outbound Costs: 2827958.88789279


2827958.88789279